# Base de datos

Funcion para crear la base de datos en sqlite 3

In [3]:
import sqlite3

# Crear la base de datos con una configuración básica
def init_db():
    conn = sqlite3.connect('hotel_reservas_basica.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS habitaciones (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        tipo_habitacion TEXT,
        estado TEXT,
        camas INTEGER,
        camas_ocupadas INTEGER
    )
    ''')
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS reservas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        id_usuario INTEGER,
        id_habitacion INTEGER,
        check_in DATE,
        check_out DATE,
        estado TEXT,
        info_pago TEXT
    )
    ''')
    
    # Insertar datos iniciales de las habitaciones (4 ordinarias y 2 compartidas)
    cursor.execute('SELECT COUNT(*) FROM habitaciones')
    if cursor.fetchone()[0] == 0:
        habitaciones_data = [
            ('King', 'disponible', 1, 0),
            ('King', 'disponible', 1, 0),
            ('Individual', 'disponible', 1, 0),
            ('Individual', 'disponible', 1, 0),
            ('Compartida', 'disponible', 4, 0),
            ('Compartida', 'disponible', 4, 0)
        ]
        cursor.executemany('INSERT INTO habitaciones (tipo_habitacion, estado, camas, camas_ocupadas) VALUES (?, ?, ?, ?)', habitaciones_data)
    
    conn.commit()
    conn.close()

In [4]:
# Ejecutar la función para inicializar la base de datos básica
init_db()

# Comprobar el contenido de las tablas
conn = sqlite3.connect('hotel_reservas_basica.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM habitaciones')
habitaciones = cursor.fetchall()
conn.close()

for i in habitaciones:
    print(i)

(1, 'King', 'disponible', 1, 0)
(2, 'King', 'disponible', 1, 0)
(3, 'Individual', 'disponible', 1, 0)
(4, 'Individual', 'disponible', 1, 0)
(5, 'Compartida', 'disponible', 4, 0)
(6, 'Compartida', 'disponible', 4, 0)
